In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
from catboost import CatBoostClassifier, Pool

In [86]:
root_dir = os.path.dirname(os.getcwd())
display(root_dir)
data_dir = os.path.join(root_dir, 'data')
display(data_dir)
train_path = os.path.join(data_dir, 'train.xlsx')
display(train_path)

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack'

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data'

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data\\train.xlsx'

In [87]:
df = pd.read_excel(train_path)
display(df.head())

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,0,20230428-6634-194809261,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,20220711-6634-144460018,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,2,20221204-16563-171020423,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,3,20230918-7491-223512699,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,4,20230529-6634-200121971,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1


In [88]:
tdf_path = os.path.join(data_dir, 'test.xlsx')
display(tdf_path)
tdf = pd.read_excel(tdf_path)
display(tdf.head())

'C:\\Users\\32233\\OneDrive\\Documents\\HSE\\AI\\projects\\PSB_hack\\data\\test.xlsx'

,Unnamed: 0,№ брони,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,0,20231129-16563-238946689,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,20221219-7491-174959103,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,2,20221211-6634-172724329,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,3,20230821-6634-212247350,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,4,20230326-6634-189784563,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1


In [89]:
tdf = tdf.drop(["Unnamed: 0", "№ брони"], axis=1)
df = df.drop(["Unnamed: 0", "№ брони"], axis=1)
display(df)
display(tdf)

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,1,25700.0,0,Внешняя система оплаты,2023-04-20 20:37:30,2023-04-20 20:39:15,2023-04-28 15:00:00,3,2023-05-01 12:00:00,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,2022-06-18 14:17:02,NaT,2022-07-11 15:00:00,2,2022-07-13 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),2022-11-14 22:59:30,NaT,2022-12-04 15:00:00,2,2022-12-06 12:00:00,Официальный сайт,Активный,Номер «Студия»,2,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),2023-09-08 15:55:53,NaT,2023-09-18 15:00:00,1,2023-09-19 12:00:00,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,2023-05-20 19:54:13,NaT,2023-05-29 15:00:00,2,2023-05-31 12:00:00,Официальный сайт,Активный,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),2023-01-07 17:45:18,NaT,2023-03-10 15:00:00,2,2023-03-12 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),2023-06-20 17:54:17,NaT,2023-06-25 15:00:00,3,2023-06-28 12:00:00,Официальный сайт,Активный,Номер «Студия»,3,4
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),2022-05-08 19:24:05,NaT,2022-06-24 15:00:00,4,2022-06-28 12:00:00,Официальный сайт,Активный,Номер «Стандарт»,2,2
26172,1,6300.0,0,Гарантия банковской картой,2022-02-19 09:55:50,2022-04-16 23:14:35,2022-04-27 15:00:00,1,2022-04-28 12:00:00,booking.com,Отмена,Номер «Стандарт»,2,3


,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),2023-11-28 10:34:43,2023-11-29 15:00:00,2,2023-12-01 12:00:00,Официальный сайт,Номер «Студия»,3,4
1,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),2022-12-12 18:30:43,2022-12-19 15:00:00,2,2022-12-21 12:00:00,Официальный сайт,Номер «Стандарт»,2,3
2,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,2022-11-25 22:03:59,2022-12-11 15:00:00,1,2022-12-12 12:00:00,Официальный сайт,Номер «Стандарт»,2,1
3,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),2023-07-18 15:45:46,2023-08-21 15:00:00,3,2023-08-24 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,2023-03-23 11:04:13,2023-03-26 15:00:00,5,2023-03-31 12:00:00,Официальный сайт,Номер «Стандарт»,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11213,1,50200.0,50200,Банк. карта [Кешбэк. МИР]: Эквайринг TravelLin...,2022-03-27 21:30:38,2022-05-07 15:00:00,2,2022-05-09 12:00:00,Официальный сайт,Номер «Студия»,4,2
11214,1,190100.0,43500,Система быстрых платежей: Эквайринг ComfortBoo...,2023-11-11 17:51:55,2024-02-16 15:00:00,5,2024-02-21 12:00:00,Официальный сайт,Коттедж с 2 спальнями,4,1
11215,1,42300.0,42300,Банк. карта: Банк Россия (банк. карта),2022-02-09 02:23:14,2022-02-09 15:00:00,1,2022-02-10 12:00:00,Официальный сайт,Коттедж с 3 спальнями,5,1
11216,1,27900.0,27900,Банк. карта: Банк Россия (банк. карта),2023-01-21 09:10:17,2023-01-28 15:00:00,1,2023-01-29 12:00:00,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1


In [90]:
df['Дата бронирования'] = df['Дата бронирования'].astype(object)
df['Заезд'] = df['Заезд'].astype(object)
df['Выезд'] = df['Выезд'].astype(object)

In [91]:
df['Дата бронирования'] = pd.to_datetime(df['Дата бронирования'])
df['Заезд'] = pd.to_datetime(df['Заезд'])
df['Выезд'] = pd.to_datetime(df['Выезд'])

In [92]:
df['Дата бронирования'] = (df['Дата бронирования'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
df['Заезд'] = (df['Заезд'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
df['Выезд'] = (df['Выезд'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [93]:
tdf['Дата бронирования'] = tdf['Дата бронирования'].astype(object)
tdf['Заезд'] = tdf['Заезд'].astype(object)
tdf['Выезд'] = tdf['Выезд'].astype(object)

In [94]:
tdf['Дата бронирования'] = pd.to_datetime(tdf['Дата бронирования'])
tdf['Заезд'] = pd.to_datetime(tdf['Заезд'])
tdf['Выезд'] = pd.to_datetime(tdf['Выезд'])

In [95]:
tdf['Дата бронирования'] = (tdf['Дата бронирования'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
tdf['Заезд'] = (tdf['Заезд'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
tdf['Выезд'] = (tdf['Выезд'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')

In [96]:
df

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Статус брони,Категория номера,Гостей,Гостиница
0,1,25700.0,0,Внешняя система оплаты,1682023050,2023-04-20 20:39:15,1682694000,3,1682942400,Яндекс.Путешествия,Отмена,Номер «Стандарт»,2,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,1655561822,NaT,1657551600,2,1657713600,Официальный сайт,Активный,Номер «Стандарт»,2,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),1668466770,NaT,1670166000,2,1670328000,Официальный сайт,Активный,Номер «Студия»,2,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),1694188553,NaT,1695049200,1,1695124800,Bronevik.com(new),Активный,Номер «Стандарт»,1,3
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,1684612453,NaT,1685372400,2,1685534400,Официальный сайт,Активный,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),1673113518,NaT,1678460400,2,1678622400,Официальный сайт,Активный,Номер «Стандарт»,2,2
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),1687283657,NaT,1687705200,3,1687953600,Официальный сайт,Активный,Номер «Студия»,3,4
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),1652037845,NaT,1656082800,4,1656417600,Официальный сайт,Активный,Номер «Стандарт»,2,2
26172,1,6300.0,0,Гарантия банковской картой,1645264550,2022-04-16 23:14:35,1651071600,1,1651147200,booking.com,Отмена,Номер «Стандарт»,2,3


In [97]:
tdf

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,23750.0,23750,Банк. карта: Банк Россия (банк. карта),1701167683,1701270000,2,1701432000,Официальный сайт,Номер «Студия»,3,4
1,1,15010.0,7505,Банк. карта: Банк Россия (банк. карта),1670869843,1671462000,2,1671624000,Официальный сайт,Номер «Стандарт»,2,3
2,1,8400.0,8400,Отложенная электронная оплата: Банк Россия (ба...,1669413839,1670770800,1,1670846400,Официальный сайт,Номер «Стандарт»,2,1
3,1,42500.0,42500,Банк. карта: Банк Россия (банк. карта),1689695146,1692630000,3,1692878400,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1
4,1,62500.0,11900,Система быстрых платежей: Эквайринг ComfortBoo...,1679569453,1679842800,5,1680264000,Официальный сайт,Номер «Стандарт»,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...
11213,1,50200.0,50200,Банк. карта [Кешбэк. МИР]: Эквайринг TravelLin...,1648416638,1651935600,2,1652097600,Официальный сайт,Номер «Студия»,4,2
11214,1,190100.0,43500,Система быстрых платежей: Эквайринг ComfortBoo...,1699725115,1708095600,5,1708516800,Официальный сайт,Коттедж с 2 спальнями,4,1
11215,1,42300.0,42300,Банк. карта: Банк Россия (банк. карта),1644373394,1644418800,1,1644494400,Официальный сайт,Коттедж с 3 спальнями,5,1
11216,1,27900.0,27900,Банк. карта: Банк Россия (банк. карта),1674292217,1674918000,1,1674993600,Официальный сайт,Апартаменты с 2 спальнями с отдельным входом,4,1


In [98]:
df['Дата отмены'] = np.where(df['Дата отмены'].isna(), 0, 1)

In [99]:
df.drop('Статус брони', axis=1, inplace=True)

In [100]:
df

,Номеров,Стоимость,Внесена предоплата,Способ оплаты,Дата бронирования,Дата отмены,Заезд,Ночей,Выезд,Источник,Категория номера,Гостей,Гостиница
0,1,25700.0,0,Внешняя система оплаты,1682023050,1,1682694000,3,1682942400,Яндекс.Путешествия,Номер «Стандарт»,2,1
1,1,24800.0,12400,Отложенная электронная оплата: Банк Россия (ба...,1655561822,0,1657551600,2,1657713600,Официальный сайт,Номер «Стандарт»,2,1
2,1,25800.0,12900,Банк. карта: Банк Россия (банк. карта),1668466770,0,1670166000,2,1670328000,Официальный сайт,Номер «Студия»,2,4
3,1,10500.0,0,Внешняя система оплаты (С предоплатой),1694188553,0,1695049200,1,1695124800,Bronevik.com(new),Номер «Стандарт»,1,3
4,1,28690.0,28690,Система быстрых платежей: Эквайринг ComfortBoo...,1684612453,0,1685372400,2,1685534400,Официальный сайт,Номер «Люкс»,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26169,1,18240.0,9120,Банк. карта: Банк Россия (банк. карта),1673113518,0,1678460400,2,1678622400,Официальный сайт,Номер «Стандарт»,2,2
26170,1,69600.0,23200,Банк. карта: Банк Россия (банк. карта),1687283657,0,1687705200,3,1687953600,Официальный сайт,Номер «Студия»,3,4
26171,1,55600.0,13900,Банк. карта: Банк Россия (банк. карта),1652037845,0,1656082800,4,1656417600,Официальный сайт,Номер «Стандарт»,2,2
26172,1,6300.0,0,Гарантия банковской картой,1645264550,1,1651071600,1,1651147200,booking.com,Номер «Стандарт»,2,3


In [101]:
cat_features = ['Номеров', 'Способ оплаты', 'Ночей', 'Источник', 'Категория номера', 'Гостей', 'Гостиница']

In [102]:
X = df.drop('Дата отмены', axis=1)
y = df['Дата отмены']

In [103]:

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=69)


In [104]:
base_model = CatBoostClassifier(cat_features=cat_features)

In [105]:
base_model.fit(X_train, y_train, verbose=0, plot=True)
prediction_proba = base_model.predict_proba(X_test)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

In [106]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_test, prediction_proba[:, 1])

0.8530350184556353

In [108]:
ans = base_model.predict_proba(tdf)[:, 1]
    
ans = pd.DataFrame(ans, columns = ['apply_promo'])
ans.to_csv('cat1.csv', index=False,header= False)